Activate venv and install packages
1. !source reasongraph/bin/activate </br>
2. !pip install "langchain>=0.3" "langgraph>=0.2" qdrant-client sentence-transformers torch pydantic python-dotenv
3. !pip install pymupdf tiktoken # Count number of tokens to check model compatibility

In [1]:
import fitz, hashlib, torch, uuid, os, sys
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models as qmodels
from pathlib import Path

/home/paul/GitHub/Private/_PKYOO-116/project__Reasoning-RAG-Framework-using-LangGraph/reasongraph/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RAGState(BaseModel):
    docs: List[str] = Field(default_factory=list)
    chunks: List[Dict[str, Any]] = Field(default_factory=list) # Includes metadata
    query: Optional[str] = None
    results: List[Dict[str, Any]] = Field(default_factory=list)
    answer: Optional[str] = None
    retry_count: int = 0

In [3]:
try:
    TOKEN_SPLIT_AVAILABLE = True
except Exception:
    TOKEN_SPLIT_AVAILABLE = False

def _hash_text(t: str) -> str:
    """페이지 중복 제거용 해시 함수"""
    return hashlib.sha256(t.strip().encode("utf-8")).hexdigest()

def load_and_chunk(state: RAGState, folder: str = "docs",
                   chunk_size_tokens: int = 350,
                   chunk_overlap_tokens: int = 50) -> RAGState:
    texts, chunks, seen_hashes = [], [], set()

    for file in Path(folder).rglob("*.pdf"):
        try:
            with fitz.open(file) as pdf:
                for i, page in enumerate(pdf, start=1):
                    text = page.get_text("text")
                    if not text or not text.strip():
                        continue

                    # Prevent page number duplication
                    h = _hash_text(text)
                    if h in seen_hashes:
                        continue
                    seen_hashes.add(h)

                    texts.append(text)

                    # Select tokenizer (token/letter)
                    if TOKEN_SPLIT_AVAILABLE:
                        splitter = TokenTextSplitter(
                            chunk_size=chunk_size_tokens,
                            chunk_overlap=chunk_overlap_tokens,
                            encoding_name="cl100k_base"
                        )
                        page_chunks = splitter.split_text(text)
                    else:
                        splitter = RecursiveCharacterTextSplitter(
                            chunk_size=800,
                            chunk_overlap=100
                        )
                        page_chunks = splitter.split_text(text)

                    # metadata saved to each chunk
                    for ci, ch in enumerate(page_chunks):
                        chunks.append({
                            "text": ch,
                            "source": file.name,
                            "page": i,
                            "chunk_index": ci
                        })

        except Exception as e:
            print(f"Failed to load PDF: {file.name} ({e})")

    if not chunks:
        print('No valid PDF chunks found in "docs" directory.')
        return state

    print(f"Loaded {len(texts)} pages and created {len(chunks)} chunks.")
    state.docs = texts
    state.chunks = chunks
    return state


In [4]:
# Use GPU to run if possible
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

Using device: cuda


In [5]:
# Specify Embedding models and load models to memory
QWEN_MODEL = "Qwen/Qwen3-Embedding-0.6B"
BGE_MODEL = "BAAI/bge-m3"
qwen = SentenceTransformer(QWEN_MODEL, device=DEVICE)
bge  = SentenceTransformer(BGE_MODEL,  device=DEVICE)

In [6]:
# Check vector dimension of each model (for qdrant collection)
QWEN_DIM = qwen.get_sentence_embedding_dimension()
BGE_DIM  = bge.get_sentence_embedding_dimension()

In [7]:
# Qdrant server connection configuration (localhost:6333)
QDRANT_HOST = os.getenv("QDRANT_HOST", "localhost")  # use default if no environment variables
QDRANT_PORT = int(os.getenv("QDRANT_PORT", "6333"))
COLLECTION  = "pkyoo_personal_docs_dualvec"                # vector collection name

In [8]:
# Qdrant client reset and connection test
try:
    client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT, timeout=60)
    _ = client.get_collections()  # get collection lists to check connection
except Exception as e:
    print("Failed to connect Qdrant. Check Qdrant Docker is running.")
    print("e.g., docker run -p 6333:6333 -v $(pwd)/qdrant_storage:/qdrant/storage qdrant/qdrant")
    raise

In [9]:
existing = [c.name for c in client.get_collections().collections]

In [10]:
# Check collection existence and create collections if not
if COLLECTION not in existing:
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config={
            "qwen": qmodels.VectorParams(size=QWEN_DIM, distance=qmodels.Distance.COSINE),
            "bge":  qmodels.VectorParams(size=BGE_DIM,  distance=qmodels.Distance.COSINE)
        }
    )
    print(f"Created collection: {COLLECTION}")
else:
    print(f"ℹCollection exists: {COLLECTION}")

ℹCollection exists: pkyoo_personal_docs_dualvec


Check http://localhost:6333/dashboard for local qdrant collections dashboard

In [11]:
# Stop the Embedding model for GPU memories. bge will still be used retrieval.
# del qwen
# torch.cuda.empty_cache()

In [ ]:
def embed_and_store(state: RAGState, batch_size=128, upsert_batch=2048) -> RAGState:
    if not state.chunks:
        print("No chunks found. Run load_and_chunk() first.")
        return state

    # Embedding only the text from chunk (No metadata)
    texts_only = [c["text"] for c in state.chunks]

    print("Encoding with Qwen3-Embedding-0.6B ...")
    qwen_vecs = qwen.encode(
        texts_only, batch_size=batch_size,
        show_progress_bar=True, normalize_embeddings=True
    )
    print("Encoding with bge-m3 ...")
    bge_vecs = bge.encode(
        texts_only, batch_size=batch_size,
        show_progress_bar=True, normalize_embeddings=True
    )

    # Points (Use metadate from chunk dict)
    points = []
    for i, (qv, bv) in enumerate(zip(qwen_vecs, bge_vecs)):
        ch = state.chunks[i]
        payload = {
            "text": ch["text"],               # Only text saved in payload
            "chunk_index": ch["chunk_index"],
            "source": ch.get("source"),
            "page": ch.get("page"),
        }
        points.append(qmodels.PointStruct(
            id=str(uuid.uuid4()),
            vector={"qwen": qv.tolist(), "bge": bv.tolist()},
            payload=payload
        ))

    print(f"Upserting {len(points)} vectors → {COLLECTION}")
    for s in range(0, len(points), upsert_batch):
        client.upsert(collection_name=COLLECTION, points=points[s:s+upsert_batch], wait=True)
    print("Upsert finished.")
    return state


In [ ]:
def retrieve_from_qdrant(state: RAGState, top_k: int = 5) -> RAGState:
    # Retrieves top-k most relevant document chunks from Qdrant
    # based on the query embedding generated by bge-m3 (RTEB model).

    if not state.query:
        print("No user query provided in state.query")
        return state

    print("Generating query embedding using bge-m3")

    # Step 1: Encode the query text using the retrieval embedding model (bge-m3)
    query_vec = bge.encode(
        [state.query],
        normalize_embeddings=True  # cosine similarity requires normalized vectors
    )[0]

    # Step 2: Search in Qdrant using the 'bge' vector field
    # The 'vector' argument must match the name used during embedding
    print(f"Searching Qdrant collection '{COLLECTION}' ...")
    hits = client.query_points(
        collection_name=COLLECTION,
        query_vector=("bge", query_vec),
        limit=top_k
    )

    # Step 3: Extract the retrieved texts (payloads), Store text, metadata, score
    results = []
    for h in hits[:top_k]:
        raw_text = h.payload.get("text", "")
        # Prevent the non-text dict saved just in case
        if isinstance(raw_text, dict):
            raw_text = raw_text.get("text", "")
        results.append({
            "text": raw_text,
            "score": getattr(h, "score", None),
            "id": getattr(h, "id", None),
            "source": h.payload.get("source"),
            "page": h.payload.get("page"),
            "chunk_index": h.payload.get("chunk_index")
        })

    # Step 4: Store retrieved chunks in RAG state
    state.results = results
    print(f"Retrieved {len(results)} chunks.")
    return state


In [14]:
import numpy as np
from sklearn.metrics import ndcg_score

In [15]:
def evaluate_retrieval_ranked(state: RAGState, top_k: int = 5,
                              relevance_threshold: float = 0.4,
                              ndcg_threshold: float = 0.6,
                              mrr_threshold: float = 0.5,
                              use_qdrant_scores: bool = False) -> str:
    # Evaluate retrieval quality using ranking metrics (nDCG@k, MRR).
    # Returns 'generate' if the retrieval quality is good enough,
    # otherwise 'rewrite' to trigger a query refinement step.

    if not state.results:
        print("No retrieved chunks to evaluate. Query rewrite required.")
        return "rewrite"

    print("Evaluating retrieval quality using ranking metrics...")
    
    # Evaluate the top_k retrieved texts
    results_k = state.results[:top_k]
    texts = [r["text"] for r in results_k]

    # Step 1: Encode the query vector using RTEB model (bge-m3)
    query_vec = bge.encode([state.query], normalize_embeddings=True)[0]

    # Step 2: Encode the retrieved chunks
    retrieved_vecs = bge.encode(texts, normalize_embeddings=True)

    # Step 3: Compute cosine similarity for each chunk
    sims = np.dot(retrieved_vecs, query_vec)

    # Step 4: Derive relevance labels (1 if above threshold, else 0)
    relevance = (sims >= relevance_threshold).astype(int)

    # Step 5: Compute ranking metrics (scores-predicted value vs relevance)
    preds = np.array([r["score"] for r in results_k]) if use_qdrant_scores else sims
    ndcg = ndcg_score([relevance], [preds])
    
    if np.any(relevance == 1):
        first_relevant_idx = int(np.argmax(relevance == 1))
        reciprocal_rank = 1.0 / (first_relevant_idx + 1)
    else:
        reciprocal_rank = 0.0

    print(f"nDCG@{top_k}: {ndcg:.3f}, MRR: {reciprocal_rank:.3f}")

    # Step 6: Decision logic
    if ndcg >= ndcg_threshold or reciprocal_rank >= mrr_threshold:
        print("Retrieval ranking is satisfactory. Proceeding to generation.")
        return "generate"
    else:
        print("Retrieval ranking is poor. Triggering query rewrite.")
        return "rewrite"


In [16]:
def rewrite_query(state: RAGState) -> RAGState:
    # Use Llama3 to rephrase the query semantically while keeping intent.
    prompt = f"""
    You are a query rewriter for a retrieval system.
    Rephrase the following query to improve retrieval quality
    without changing its meaning or intent.

    Query:
    "{state.query}"
    """

    new_query = llama3.generate(prompt)  # pseudo-call
    new_query = (new_query or "").strip()

    print(f"""
          Rewritten query (attempt {state.retry_count + 1}):
          "{new_query}"
          """)
    state.query = new_query
    return state

Loop stops when number of rewrites = 5 to limit the response time and endless querying.

In [17]:
def retrieval_loop(state: RAGState, max_retries: int = 5, top_k: int = 5) -> RAGState:
    # Full retrieval + evaluation + rewrite loop to prevent hallucination.
    while state.retry_count <= max_retries:
        print(f"\n[Attempt {state.retry_count + 1}] Retrieving and evaluating...")
        
        # Step 1: Retrieve
        state = retrieve_from_qdrant(state, top_k=top_k)

        # Step 2: Evaluate
        result = evaluate_retrieval_ranked(state, top_k=top_k)

        if result == "generate":
            print("Retrieval sufficient to proceed LLM generation.")
            return state  # Pass to generation stage

        # Step 3: If evaluation fails, rewrite query
        if state.retry_count < max_retries:
            print("Rewriting query and retrying retrieval...")
            state = rewrite_query(state)
            state.retry_count += 1
        else:
            print("Retrieval failed after max attempts. No relevant info found.")
            state.answer = "I'm sorry, but I couldn’t find relevant information about Paul from the database."
            return state
    return state

In [18]:
from langchain_community.llms import Ollama
import textwrap
llama3 = Ollama(model="llama3", temperature=0.2)

/tmp/ipykernel_24046/309009186.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llama3 = Ollama(model="llama3", temperature=0.2)


In [ ]:
def generate_answer(state: RAGState, model: str = "llama3", max_context: int = 5) -> RAGState:
    """
    Generate an answer using retrieved chunks as context.
    Uses a local Llama3 model (via Ollama) for answer generation.
    """

    # Step 1: Retrieve top-k context chunks (from previous retrieval step)
    if not state.results:
        print("No retrieval results found. Run retrieval_loop() first.")
        state.answer = "I'm sorry, but I couldn’t find relevant information about Paul from the database."
        return state

    # Limit context to top N results for efficiency
    # Prevent dict that is not text
    top_contexts_raw = [r.get("text", "") for r in state.results[:max_context]]
    top_contexts = [
        (c.get("text", "") if isinstance(c, dict) else c) for c in top_contexts_raw
    ]
    combined_context = "\n\n".join(top_contexts)

    # Step 2: Build a system + user prompt
    prompt = textwrap.dedent(f"""
    You are a precise and concise AI assistant specialized in retrieval-augmented generation.
    Use the following context extracted from trusted documents to answer the user's query accurately.
    If the context does not contain sufficient information, clearly say so without hallucinating.

    --- Context ---
    {combined_context}

    --- User Query ---
    {state.query}

    --- Instruction ---
    1. Base your answer only on the given context.
    2. DO NOT invent facts not present in the documents(context).
    3. If unsure, say "I'm sorry, but I couldn’t find relevant information about Paul from the database."
    4. Return your final answer clearly and concisely.
    """)

    print("Generating answer...")
    try:
        response = model.invoke(prompt)
        answer_text = response.strip()
    except Exception as e:
        print(f"Failed to connect GenAI Model: {e}")
        answer_text = "Error: failed to generate answer due to model connection or runtime issue."

    # Step 3: Store generated answer
    state.answer = answer_text

    # Step 4: Log summary
    print(" ")
    print("\n====== Generated Answer ======")
    print(" ")
    print(answer_text)
    print(" ")
    print("================================\n")
    print(" ")

    return state


### GenAI generated Answer Evaluation
1. Sementic Evaluation using DeepEval Faithfulness and Answer Relevancy
  - Faithfulness: Is answer based on the context retrieved by the retrieval model?
  - Answer Relevancy: Is answer relevant to the user query(question)?

2. Liguistic Evaluation using DeepEval and custom metrics
  - Grammar: Grammar Evaluation
  - Fluency: Fluency Evaluation
  - Coherence: Evaluating consistency of logical structure between sentences
  - Conciseness: Evaluating the unnecessary repeatition
  - Toxicity and Bias: Ethical use of words and sentences Evaluation

In [23]:
from deepeval.models import OllamaModel
from deepeval.metrics import FaithfulnessMetric, AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

In [25]:
eval_model = OllamaModel(model="llama3")

faith_metric = FaithfulnessMetric(model=eval_model, threshold=0.7)
relev_metric = AnswerRelevancyMetric(model=eval_model, threshold=0.7)

In [26]:
def evaluate_answer_tier1(state: RAGState,
                          faith_thresh: float = 0.7,
                          relev_thresh: float = 0.7) -> str:
    """
    Tier 1 evaluation:
    - Checks semantic validity (Faithfulness + Answer Relevancy)
    - Returns 'pass' or 'rewrite'
    """

    if not state.answer or not state.results:
        print("No generated answer or context found for Tier 1 evaluation.")
        return "rewrite"

    context = "\n".join([r["text"] for r in state.results])

    # Define evaluation test case
    test_case = LLMTestCase(
        input=state.query,
        actual_output=state.answer,
        retrieval_context=[context],
        expected_output=None
    )

    print("Evaluating Tier 1 - Faithfulness & Relevancy...")
    results = evaluate(test_cases=[test_case], metrics=[faith_metric, relev_metric])

    # Extract metric scores
    faith_score = results[0][faith_metric.name]["score"]
    relev_score = results[0][relev_metric.name]["score"]

    print(f"Faithfulness: {faith_score:.3f} | Relevancy: {relev_score:.3f}")

    if faith_score >= faith_thresh and relev_score >= relev_thresh:
        print("Passed Tier 1 - Semantic Evaluation")
        return "pass"
    else:
        print("Failed Tier 1 - Sending back to rewrite.")
        return "rewrite"


In [27]:
from deepeval.metrics import BaseMetric

In [28]:
class GrammarMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.75):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Grammar Quality"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Evaluate the grammar and sentence correctness of this answer.
        Give a score between 0 (poor) and 1 (perfect grammar).
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

class FluencyMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.75):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Fluency"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Rate the fluency and naturalness of this answer from 0 to 1.
        Avoid judging content, only evaluate smoothness and readability.
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

class CoherenceMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.75):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Coherence"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Evaluate the logical coherence and flow of the following answer.
        Give a score from 0 (disjointed) to 1 (highly coherent).
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

class ConcisenessMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.6):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Conciseness"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Evaluate how concise this answer is.
        Penalize redundancy or overly long explanations.
        Score from 0 (verbose) to 1 (clear and concise).
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

class ToxicityMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.2):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Toxicity"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Detect any offensive, hateful, or toxic language in the following answer.
        Give a score from 0 (completely safe) to 1 (very toxic).
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

class BiasMetric(BaseMetric):
    def __init__(self, model="llama3", threshold=0.3):
        super().__init__()
        self.model = model
        self.threshold = threshold
        self.name = "Bias"

    def measure(self, question, answer, context=None):
        prompt = f"""
        Evaluate if this answer contains biased or discriminatory language.
        Give a score from 0 (neutral) to 1 (biased).
        Answer:
        {answer}
        """
        score = self.call_model(prompt)
        return float(score)

# --- Tier 2 Evaluation Logic ---
def evaluate_answer_tier2(state: RAGState) -> str:
    """
    Tier 2 evaluation:
    - Checks Grammar, Fluency, Coherence, Conciseness, Toxicity, Bias
    - Returns 'final' or 'rewrite_once'
    """

    if not state.answer:
        print("No answer found for Tier 2 evaluation.")
        return "rewrite_once"

    print("Evaluating Tier 2 - linguistic/ethical evaluation...")

    metrics = [
        GrammarMetric(), FluencyMetric(), CoherenceMetric(),
        ConcisenessMetric(), ToxicityMetric(), BiasMetric()
    ]

    context = "\n".join([r["text"] for r in state.results]) if state.results else ""
    question = state.query or "General query"

    # Evaluate
    scores = {}
    for m in metrics:
        try:
            score = m.measure(question, state.answer, context)
            scores[m.name] = score
        except Exception as e:
            scores[m.name] = 0.0
            print(f"Metric {m.name} failed: {e}")

    # Log scores
    for k, v in scores.items():
        print(f"{k}: {v:.3f}")

    # Determine pass/fail
    if (scores["Grammar Quality"] >= 0.75 and
        scores["Fluency"] >= 0.75 and
        scores["Coherence"] >= 0.75 and
        scores["Conciseness"] >= 0.6 and
        scores["Toxicity"] <= 0.2 and
        scores["Bias"] <= 0.3):
        print("Passed Tier 2 - Linguistic/Ethical Evaluation")
        return "final"
    else:
        print("Failed Tier 2 - Sending back to rewrite.")
        return "rewrite_once"


In [29]:
def full_answer_evaluation(state: RAGState,
                           max_semantic_retries: int = 5,
                           max_linguistic_retries: int = 3) -> RAGState:
    """
    Full evaluation pipeline:
    - Tier 1 + Tier 2
    - Adaptive retry: max total rewrites = 8
    """

    # Tier 1 - Semantic evaluation
    while state.retry_count < max_semantic_retries:
        print(f"\n[Semantic Eval Attempt {state.retry_count + 1}]")

        tier1_result = evaluate_answer_tier1(state)
        if tier1_result == "pass":
            print("Passed Tier 1. Moving to Tier 2.")
            break

        # Rewrite on fail
        state = rewrite_answer(state)
        state.retry_count += 1

    else:
        print("Tier 1 failed after max retries.")
        state.answer = (
            "I'm sorry, but I couldn’t generate a factually accurate and relevant answer. Could you try again?"
        )
        return state

    # Tier 2 - Linguistic/Ethical evaluation
    linguistic_retries = 0
    while linguistic_retries < max_linguistic_retries:
        print(f"\n[Linguistic Eval Attempt {linguistic_retries + 1}]")

        tier2_result = evaluate_answer_tier2(state)
        if tier2_result == "final":
            print("Passed Tier 2. Answer fully verified.")
            return state

        # Rewrite & adjust retry counters
        state = rewrite_answer(state)
        linguistic_retries += 1

        # Deduct semantic retry budget
        if state.retry_count < max_semantic_retries:
            state.retry_count += 1
            print(f"Deducted 1 semantic retry (used {state.retry_count}/{max_semantic_retries})")

        # Abort if total rewrite exceeds 8
        if state.retry_count >= max_semantic_retries:
            print("Answer generation total attempts exceeded 8. Aborting.")
            state.answer = (
                "I'm sorry, but I couldn’t refine the answer further without risking factual distortion. Could you try again?"
            )
            return state

    print("Minor linguistic issues remain. Accepting best-effort answer.")
    return state


In [ ]:
# Initialize State
state = RAGState()
state.query = input("Enter your question: ").strip()

# Load & Chunk PDFs (only once unless documents change)
print("\n--- Loading and Chunking Documents ---")
state = load_and_chunk(state)

# Embed and Store in Qdrant (only once after load)
print("\n--- Embedding and Storing Chunks in Qdrant ---")
state = embed_and_store(state)

# release Qwen from GPU to free memory
print("Deleting MTEB model from cache")
del qwen
torch.cuda.empty_cache()

# Retrieval Loop — uses bge-m3 for retrieval, evaluation & rewriting if needed
print("\n--- Starting Retrieval Loop ---")
state = retrieval_loop(state)

# Answer Generation using Llama3
print("\n--- Generating Answer ---")
state = generate_answer(state)

# Dual-layer Evaluation (DeepEval Tier1 + Custom Tier2)
print("\n--- Evaluating Generated Answer ---")
state = full_answer_evaluation(state)

# Final Output (Answer to the question of user query)
print("\n=====================================================")
print("")
print(state.answer)
print("")
print("=====================================================\n")



--- Loading and Chunking Documents ---
Loaded 21 pages and created 37 chunks.

--- Embedding and Storing Chunks in Qdrant ---
Encoding with Qwen3-Embedding-0.6B ...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


Encoding with bge-m3 ...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]
/tmp/ipykernel_24046/2927138612.py:20: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(


Upserting 37 vectors → pkyoo_personal_docs_dualvec
Upsert finished.

--- Starting Retrieval Loop ---

[Attempt 1] Retrieving and evaluating...
Generating query embedding using bge-m3
Searching Qdrant collection 'pkyoo_personal_docs_dualvec' ...
Retrieved 5 chunks.
Evaluating retrieval quality using ranking metrics...
nDCG@5: 1.000, MRR: 1.000
Retrieval ranking is satisfactory. Proceeding to generation.
Retrieval sufficient to proceed LLM generation.

--- Generating Answer ---


TypeError: sequence item 0: expected str instance, dict found